# Revised case normalization for Winterthur 2018

Notes:

- no 'Admno' or 'Fall Nummer' to be matched to DB

In [1]:
import sys
sys.path.insert(0, '/home/jovyan/work')

from src.revised_case_normalization.notebook_functions.global_configs import *
from src.revised_case_normalization.notebook_functions.normalize import normalize_ksw_2018_2019
from src.revised_case_normalization.notebook_functions.revise import revise, get_sociodemographics_for_hospital_year
from src.revised_case_normalization.notebook_functions.group import group
from src.revised_case_normalization.notebook_functions.update_db import update_db

In [2]:
file_info = FileInfo(
         os.path.join(ROOT_DIR, 'raw_data/Winterthur.xlsx'),
         'KSW',
         '2018',
         'Änderungen Winterthur 2018')

print(file_info)



FileInfo(path='/home/jovyan/work/src/revised_case_normalization/raw_data/Winterthur.xlsx', hospital_name_db='KSW', year='2018', sheet='Änderungen Winterthur 2018')


In [3]:
revised_cases_df = normalize_ksw_2018_2019(file_info)
revised_cases_df

2022-11-14 09:25:13.261 | INFO     | src.revised_case_normalization.notebook_functions.normalize:normalize_ksw_2018_2019:141 - Read 193 cases for KSW 2018
2022-11-14 09:25:13.266 | INFO     | src.revised_case_normalization.notebook_functions.normalize:normalize_ksw_2018_2019:170 - TYPES:
tranche                                                 string
datum                                                   string
case_id                                                 object
patient_id                                              object
kkik                                                    string
gender                                                  string
age_years                                                int64
bfs_code                                                string
duration_of_stay                                         int64
pflegetage neu                                          string
pccl                                                     int64
pccl neu          

,case_id,patient_id,gender,age_years,duration_of_stay,case_id_norm,old_pd,primary_diagnosis,bfs_code,pccl,drg,added_icds,removed_icds,added_chops,removed_chops
0,900362200,120A84ED0B5D412F,W,54,8,900362200,C795,C795,M200,3,I10B,[M4957],[],[],[]
1,900449400,2833C0B8F3FAF2B2,W,80,12,900449400,N1793,N1793,M100,3,L60D,[G2090],[],[],[]
2,900404910,220435AE84965999,M,79,3,900404910,S3702,S3702,M200,3,L09B,[S3683],[S3681],[],[]
3,900454320,472E599EDB7090A6,W,66,2,900454320,R410,R410,M100,3,B64B,[L8918],[L8908],[],[]
4,900475230,B127DAEB25A7546F,W,58,9,900475230,C56,C56,M200,3,N60B,"[I1091, K658, Y849]",[I1090],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,900328975,0297730,W,83,5,900328975,C160,C160,M100,3,G60B,"[D630, E1191, C772]","[D500, E1190]",[],[]
189,900359871,0389213,M,87,9,900359871,N390,A491,M100,3,L63D,[B952],[],[],[]
190,900405823,0683082,W,87,8,900405823,I5001,I5001,M100,3,F62C,[R64],[],[],[]
191,900478805,0449127,W,92,11,900478805,K922,K922,M100,3,G48C,[D62],[],[],[]


# Match revised DtoD data with the BfS data from the database

In [4]:
cols_to_join = list(VALIDATION_COLS)
cols_to_join.remove(CASE_ID_COL)
cols_to_join.append(NORM_CASE_ID_COL)
# Ignore the patient ID in this dataset, because it is encrypted
cols_to_join.remove(PATIENT_ID_COL)

revised_cases, unmatched = revise(file_info, revised_cases_df, validation_cols=cols_to_join)
revised_cases.head()

2022-11-14 09:25:15.206 | INFO     | src.service.bfs_cases_db_service:get_sociodemographics_for_hospital_year:90 - Read 27501 rows from the DB, for the hospital 'KSW' in 2018
2022-11-14 09:25:15.228 | WARNING  | src.revised_case_normalization.notebook_functions.revise:revise:47 - 193 rows could not be matched, given ['age_years', 'case_id_norm', 'duration_of_stay', 'gender']


KeyError: "None of [Index(['revision_id', 'code'], dtype='object')] are in the [columns]"

In [ ]:
if unmatched.shape[0] > 0:
    display(unmatched)

In [ ]:
revisions_update, diagnoses_update, procedures_update = group(revised_cases)
revisions_update[REVISION_DATE_COL] = '2022-12-31'

In [ ]:
revisions_update

In [ ]:
update_db(revisions_update, diagnoses_update, procedures_update)